In [1]:
from trainer import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8405,107.39,ak,s,as,c,d,aa,d,q,...,1,0,0,0,0,0,0,0,0,0
4205,8406,108.77,j,o,t,d,d,aa,h,h,...,0,1,0,0,0,0,0,0,0,0
4206,8412,109.22,ak,v,r,a,d,aa,g,e,...,0,0,1,0,0,0,0,0,0,0
4207,8415,87.48,al,r,e,f,d,aa,l,u,...,0,0,0,0,0,0,0,0,0,0


In [3]:
train_df, eval_df = train_test_split(train_df, test_size=0.2, random_state=42)
train_df

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
1006,2011,88.96,m,v,as,c,d,ag,k,x,...,0,0,1,0,0,0,0,0,0,0
1840,3690,89.90,n,s,as,d,d,ae,g,s,...,1,0,0,0,0,0,0,0,0,0
3792,7597,92.59,f,c,m,c,d,v,i,e,...,0,0,1,0,0,0,0,0,0,0
152,322,108.84,j,aa,g,d,d,i,i,e,...,0,1,0,0,0,0,0,0,0,0
1557,3103,111.15,ay,i,as,c,d,ad,l,k,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3444,6879,109.42,j,i,as,c,d,r,l,r,...,0,0,1,0,0,0,0,0,0,0
466,898,78.25,az,y,e,c,d,d,j,j,...,0,0,0,0,0,0,1,0,0,0
3092,6214,92.18,y,w,ae,c,d,q,i,c,...,1,0,0,0,0,0,0,0,0,0
3772,7558,91.92,y,r,ak,f,d,v,i,b,...,0,0,0,0,0,0,0,0,0,0


In [4]:
from sklearn.metrics import r2_score

r2 = Scorer(name="r2",
            scorer=r2_score,
            greater_is_better=True,
            extra_params = {})

preprocessor = PreprocessingTool(val_folds = True,
                                 n_folds = 5,
                                 seed = 42,
                                 prob_type="regression")

train_dataset = TabularDataset(train_df,
                               label = "y",
                               preprocessor = preprocessor,
                               type = "train")
eval_dataset = TabularDataset(eval_df,
                              label = "y",
                              preprocessor = preprocessor,
                              type = "eval")
test_dataset = TabularDataset(test_df,
                              preprocessor = preprocessor,
                              type = "infer")

trainer = Trainer(train_dataset,
                  eval_dataset = eval_dataset,
                  eval_metric = r2,
                  models = ["LGB", "XGB", "CAT"],
                  early_stopping_rounds = 100,
                  of_mitigation_level = 0.2,
                  use_gpu = True,
                  use_cuda = True,
                  n_trials = 1000,
                  timeout = 3600,
                  seed = 42,
                  select_top = 3,
                  train_meta = True)

trainer.train()

LABEL: y
Problem type: regression. 
-----------------------------------------------------------------------------------------------------------------------------------------
IMPUTER:

Done. 
-----------------------------------------------------------------------------------------------------------------------------------------
DTYPES DETECTOR:


Found 2 unique raw np.dtype(s): [dtype('int64') dtype('O')].
Dropped 1 column(s) with index ID / non-categorical text features.
Converted 0 column(s) to numeric types.
Remaining features: 368 int feat(s), 0 float feat(s), 8 categorical feat(s).
-----------------------------------------------------------------------------------------------------------------------------------------
5-FOLD STRATIFIEDKFOLDREG TRAIN-VAL SPLITTER:

Done.
-----------------------------------------------------------------------------------------------------------------------------------------
SCALER:

Done.
---------------------------------------------------------------

  0%|          | 0/1000 [00:00<?, ?it/s]

Trial 0 finished with values: {'mean_train_r2': 0.488254275455422, 'mean_val_r2': 0.45925195458726453, 'optimized_metric': 0.45345149041363303},
and parameters: {'metric': 'None', 'learning_rate': 0.009089922354741157, 'n_estimators': 287, 'max_depth': 12, 'num_leaves': 144, 'feature_fraction': 0.635115823765412, 'bagging_fraction': 0.5035078908389963, 'lambda_l1': 0.0012605905632811774, 'lambda_l2': 4.736355058401313, 'min_child_samples': 72, 'boost_from_average': True, 'verbosity': -1, 'n_estimators_': 233}.

Trial 1 finished with values: {'mean_train_r2': 0.5745903880438783, 'mean_val_r2': 0.5512006145052394, 'optimized_metric': 0.5465226597975116},
and parameters: {'metric': 'None', 'learning_rate': 0.02198344509605811, 'n_estimators': 210, 'max_depth': 17, 'num_leaves': 147, 'feature_fraction': 0.8445149379764622, 'bagging_fraction': 0.8666692777096867, 'lambda_l1': 0.26708993515776386, 'lambda_l2': 1.2544498874647192, 'min_child_samples': 88, 'boost_from_average': False, 'verbosi

,id,mean_train_r2,mean_val_r2,optimized_metric,params_bagging_fraction,params_boost_from_average,params_feature_fraction,params_lambda_l1,params_lambda_l2,params_learning_rate,...,params_min_child_samples,params_n_estimators,params_num_leaves,params_verbosity,params_n_estimators_,datetime_start,datetime_complete,duration,state,model
0,LGB_48,0.585026,0.562691,0.558225,0.886774,False,0.654218,6.635652,0.667823,0.018548,...,64,876,206,-1,309,2024-09-11 01:40:48.481859,2024-09-11 01:41:04.531550,0 days 00:00:16.049691,COMPLETE,LGBMRegressor(bagging_fraction=0.8867738962035...
1,LGB_28,0.589781,0.562850,0.557464,0.859717,False,0.626979,8.567082,2.674302,0.016639,...,61,848,180,-1,360,2024-09-11 01:33:00.494214,2024-09-11 01:33:26.377709,0 days 00:00:25.883495,COMPLETE,LGBMRegressor(bagging_fraction=0.8597169579608...
2,LGB_16,0.595508,0.563700,0.557339,0.757184,False,0.709684,0.680374,6.174017,0.018375,...,60,897,91,-1,342,2024-09-11 01:28:28.966515,2024-09-11 01:28:50.409684,0 days 00:00:21.443169,COMPLETE,LGBMRegressor(bagging_fraction=0.7571835327666...


------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ 


OPTUNING XGB MODEL...



...

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ 


Generated ensemble model. To use it, call .predict() / .predict_proba() method on trainer object.


VotingRegressor(estimators=[('XGB_40',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=0.8121086989559163,
                                          device='cuda',
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=0.47042627005230425,
                                          grow_policy=None,
                                          importance_type=None,
                                          interact...
                                           feature_fraction=0.7096843998570975,
                                           lambda_l1=0.6803742388458807,
                                           lambda_l2=6.1740173735226564,
                                           learning_rate=0.018374813949189543,
                                           max_depth=9, metric='None',
                                           min_child_samples=60,
                                           n_estimators=342, num_leaves=91,
                                           seed=42, verbosity=-1))],
                weights=0    0.111676
1    0.111649
2    0.111572
3    0.111397
4    0.111220
5    0.111208
6    0.110534
7    0.110384
8    0.110359
Name: optimized_metric, dtype: float64)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ 


External Evaluation LEADERBOARD:


,id,model,custom_r2,mae,mse,rmse,r2
0,CAT_19,<catboost.core.CatBoostRegressor object at 0x0...,0.602980,5.236466,61.796068,7.861048,0.602980
1,MetaLearner,"ElasticNet(alpha=4.39049150145519, l1_ratio=2....",0.601048,5.282649,62.096897,7.880158,0.601048
2,VotingEnsemble,"VotingRegressor(estimators=[('XGB_40',\n ...",0.599553,5.248590,62.329619,7.894911,0.599553
3,XGB_26,"XGBRegressor(base_score=None, booster=None, ca...",0.597859,5.273084,62.593248,7.911589,0.597859
4,CAT_10,<catboost.core.CatBoostRegressor object at 0x0...,0.597774,5.266428,62.606376,7.912419,0.597774
5,CAT_13,<catboost.core.CatBoostRegressor object at 0x0...,0.597080,5.274904,62.714499,7.919249,0.597080
6,XGB_40,"XGBRegressor(base_score=None, booster=None, ca...",0.596226,5.276679,62.847355,7.927632,0.596226
7,XGB_35,"XGBRegressor(base_score=None, booster=None, ca...",0.595604,5.279985,62.944213,7.933739,0.595604
8,LGB_28,LGBMRegressor(bagging_fraction=0.8597169579608...,0.593423,5.284391,63.283752,7.955109,0.593423
9,LGB_16,LGBMRegressor(bagging_fraction=0.7571835327666...,0.593113,5.298821,63.331922,7.958136,0.593113


------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ 


Retraining best found models on whole available data...
Done. Trainer is ready for inference.
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ 




In [18]:
import dill
with open("tr.pkl", "wb") as f:
    dill.dump(trainer, f)

In [19]:
with open("tr.pkl", "rb") as f:
    new_tr = dill.load(f)

In [21]:
processed_test_data = test_dataset.process()
processed_test_data

,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,...,X8_p,X8_q,X8_r,X8_s,X8_t,X8_u,X8_v,X8_w,X8_x,X8_y
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4206,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4207,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
preds = trainer.predict(processed_test_data, mode="ensemble")
preds

array([ 78.72376718,  94.46495028,  78.8818638 , ...,  92.88526377,
       111.10183589,  92.96680061])

In [10]:
submission = pd.DataFrame({
    "ID": test_df["ID"],
    "y": preds
})
submission

,ID,y
0,1,78.723767
1,2,94.464950
2,3,78.881864
3,4,77.956778
4,5,112.631084
...,...,...
4204,8410,103.391248
4205,8411,93.705470
4206,8413,92.885264
4207,8414,111.101836


In [11]:
submission.to_csv("submission_meta.csv", index=False)

a
b
d


In [43]:
lista

['a', 'b', 'c']